# Lab 12 - Bike Stats
#### Vijayeta Rangarajan
#### 27th May 2020


In [3]:
import numpy as np
import pandas as pd
import re

In [4]:
wf = pd.read_csv('weather.csv')

In [5]:
trip = pd.read_csv('trip.csv', error_bad_lines=False)

b'Skipping line 50794: expected 12 fields, saw 20\n'


In [6]:
st = pd.read_csv('station.csv')

In [7]:
st.head()

,station_id,name,lat,long,install_date,install_dockcount,modification_date,current_dockcount,decommission_date
0,BT-01,3rd Ave & Broad St,47.618418,-122.350964,10/13/2014,18,NaN,18,NaN
1,BT-03,2nd Ave & Vine St,47.615829,-122.348564,10/13/2014,16,NaN,16,NaN
2,BT-04,6th Ave & Blanchard St,47.616094,-122.341102,10/13/2014,16,NaN,16,NaN
3,BT-05,2nd Ave & Blanchard St,47.613110,-122.344208,10/13/2014,14,NaN,14,NaN
4,CBD-03,7th Ave & Union St,47.610731,-122.332447,10/13/2014,20,NaN,20,NaN


In [8]:
wf.head()

,Date,Max_Temperature_F,Mean_Temperature_F,Min_TemperatureF,Max_Dew_Point_F,MeanDew_Point_F,Min_Dewpoint_F,Max_Humidity,Mean_Humidity,Min_Humidity,...,Mean_Sea_Level_Pressure_In,Min_Sea_Level_Pressure_In,Max_Visibility_Miles,Mean_Visibility_Miles,Min_Visibility_Miles,Max_Wind_Speed_MPH,Mean_Wind_Speed_MPH,Max_Gust_Speed_MPH,Precipitation_In,Events
0,10/13/2014,71,62.0,54,55,51,46,87,68,46,...,29.79,29.65,10,10,4,13,4,21,0.00,Rain
1,10/14/2014,63,59.0,55,52,51,50,88,78,63,...,29.75,29.54,10,9,3,10,5,17,0.11,Rain
2,10/15/2014,62,58.0,54,53,50,46,87,77,67,...,29.71,29.51,10,9,3,18,7,25,0.45,Rain
3,10/16/2014,71,61.0,52,49,46,42,83,61,36,...,29.95,29.81,10,10,10,9,4,-,0.00,Rain
4,10/17/2014,64,60.0,57,55,51,41,87,72,46,...,29.78,29.73,10,10,6,8,3,-,0.14,Rain


In [9]:
trip.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,birthyear
0,431,10/13/2014 10:31,10/13/2014 10:48,SEA00298,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1960.0
1,432,10/13/2014 10:32,10/13/2014 10:48,SEA00195,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1970.0
2,433,10/13/2014 10:33,10/13/2014 10:48,SEA00486,883.831,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,1988.0
3,434,10/13/2014 10:34,10/13/2014 10:48,SEA00333,865.937,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,1977.0
4,435,10/13/2014 10:34,10/13/2014 10:49,SEA00202,923.923,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1971.0


### What is the average trip duration for a borrowed bicycle?

In [10]:
avg_trip_duration = trip['tripduration'].mean()
avg_trip_duration = int(avg_trip_duration)
avg_trip_duration

1178

### What’s the most common age of a bicycle-sharer?

In [11]:
most_common_year = trip['birthyear'].mode()
from datetime import datetime
current_year = datetime.now().year
most_common_age = current_year - most_common_year.iloc[0]
most_common_age

33.0

### Find the average precipitation per month, and the median precipitation
* divide the data by month (ie take out the month at the beginning of the date)
* then group the data by the month
* then find the average precipitation for each month


In [12]:
wf['Month'] = wf.apply(lambda mo: mo['Date'].split("/")[0], axis = 1)
wf.head()

,Date,Max_Temperature_F,Mean_Temperature_F,Min_TemperatureF,Max_Dew_Point_F,MeanDew_Point_F,Min_Dewpoint_F,Max_Humidity,Mean_Humidity,Min_Humidity,...,Min_Sea_Level_Pressure_In,Max_Visibility_Miles,Mean_Visibility_Miles,Min_Visibility_Miles,Max_Wind_Speed_MPH,Mean_Wind_Speed_MPH,Max_Gust_Speed_MPH,Precipitation_In,Events,Month
0,10/13/2014,71,62.0,54,55,51,46,87,68,46,...,29.65,10,10,4,13,4,21,0.00,Rain,10
1,10/14/2014,63,59.0,55,52,51,50,88,78,63,...,29.54,10,9,3,10,5,17,0.11,Rain,10
2,10/15/2014,62,58.0,54,53,50,46,87,77,67,...,29.51,10,9,3,18,7,25,0.45,Rain,10
3,10/16/2014,71,61.0,52,49,46,42,83,61,36,...,29.81,10,10,10,9,4,-,0.00,Rain,10
4,10/17/2014,64,60.0,57,55,51,41,87,72,46,...,29.73,10,10,6,8,3,-,0.14,Rain,10


In [13]:
avg_monthly_precipitation = wf[["Precipitation_In", "Month"]].groupby("Month").mean().sort_values("Precipitation_In", ascending = False)
avg_monthly_precipitation

,Precipitation_In
Month,
12,0.236290
10,0.189000
11,0.187833
2,0.168421
3,0.156935
1,0.143548
4,0.051333
9,0.041000
6,0.030500


In [14]:
median_precipitation = wf["Precipitation_In"].median()
median_precipitation

0.0

### What’s the average number of bikes at a given bike station?

In [15]:
st.head()

,station_id,name,lat,long,install_date,install_dockcount,modification_date,current_dockcount,decommission_date
0,BT-01,3rd Ave & Broad St,47.618418,-122.350964,10/13/2014,18,NaN,18,NaN
1,BT-03,2nd Ave & Vine St,47.615829,-122.348564,10/13/2014,16,NaN,16,NaN
2,BT-04,6th Ave & Blanchard St,47.616094,-122.341102,10/13/2014,16,NaN,16,NaN
3,BT-05,2nd Ave & Blanchard St,47.613110,-122.344208,10/13/2014,14,NaN,14,NaN
4,CBD-03,7th Ave & Union St,47.610731,-122.332447,10/13/2014,20,NaN,20,NaN


In [19]:
station_dockcounts = st[st["decommission_date"].isnull()]
st_mean = int(station_dockcounts["current_dockcount"].mean())
st_mean

17

### When a bike station is modified, is it more likely that it’ll lose bikes or gain bikes? How do you know? - I am unsure how to answer this question

In [28]:
mod_stations = st[['modification_date','install_dockcount','current_dockcount']].dropna()
bike_status = mod_stations.current_dockcount.sum() - mod_stations.install_dockcount.sum()
bike_status

-64

### Find the average precipitation per year

In [99]:
wf['Year'] = wf.apply(lambda mo: mo['Date'].split("/")[2], axis = 1)
avg_yearly_precipitation = wf[["Precipitation_In", "Year"]].groupby("Year").mean().sort_values("Precipitation_In", ascending = False)
avg_yearly_precipitation

,Precipitation_In
Year,
2014,0.180750
2015,0.100740
2016,0.086721


In [101]:
trip.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,birthyear
0,431,10/13/2014 10:31,10/13/2014 10:48,SEA00298,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1960.0
1,432,10/13/2014 10:32,10/13/2014 10:48,SEA00195,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1970.0
2,433,10/13/2014 10:33,10/13/2014 10:48,SEA00486,883.831,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,1988.0
3,434,10/13/2014 10:34,10/13/2014 10:48,SEA00333,865.937,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,1977.0
4,435,10/13/2014 10:34,10/13/2014 10:49,SEA00202,923.923,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1971.0


### What month was the highest average ride in Seattle?

In [110]:
trip['Month'] = trip.apply(lambda yr:yr['starttime'].split("/")[0],axis = 1)
trip.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,birthyear,Month
0,431,10/13/2014 10:31,10/13/2014 10:48,SEA00298,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1960.0,10
1,432,10/13/2014 10:32,10/13/2014 10:48,SEA00195,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1970.0,10
2,433,10/13/2014 10:33,10/13/2014 10:48,SEA00486,883.831,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,1988.0,10
3,434,10/13/2014 10:34,10/13/2014 10:48,SEA00333,865.937,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,1977.0,10
4,435,10/13/2014 10:34,10/13/2014 10:49,SEA00202,923.923,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1971.0,10


In [112]:
max_trip_duration = trip[['bikeid','starttime','tripduration',"Month"]].groupby('Month').mean().sort_values('tripduration', ascending = False)
max_trip_duration

,tripduration
Month,
7,1414.474540
6,1324.772697
8,1323.178564
5,1305.628828
4,1257.183812
9,1192.563013
3,1155.070784
2,1074.986081
10,1009.585596


### What is the average distance traveled by females vs males

In [121]:
female_ride = trip[['gender', 'tripduration']]
female_ride = female_ride[female_ride['gender']=='Female']
female_ride_average = female_ride['tripduration'].mean()
female_ride_average

706.723239444119

In [125]:
male_ride = trip[['gender', 'tripduration']]
male_ride = male_ride[male_ride['gender']=='Male']
male_ride_average = male_ride['tripduration'].mean()
male_ride_average

564.8808333855042

In [ ]:
female_ride_average - male_ride_average

In [29]:
def test():

    def assert_equal(actual,expected):
        assert actual == expected, f"Expected {expected} but got {actual}"
    assert_equal(avg_trip_duration,1178)
    assert_equal(most_common_age, 33)
    assert_equal(median_precipitation,0)
    assert_equal(avg_monthly_precipitation.iloc[0].name, '12')
    assert_equal(st_mean,17)
    assert_equal(bike_status, -64)
#     assert_equal(avg_station_bikes.iloc[0].name,'DPD')
#     assert_equal(avg_yearly_precipitation.iloc[0].name,'2014')
#     assert_equal(max_trip_duration.iloc[0].name,'7')
    print("success")
    
test()

success
